Criar um conjunto de filmes que possa ser oferecido a determinado usuário

# 0. Importar as bibliotecas

In [22]:
import pandas as pd
import matplotlib.pyplot as plt #plotar gráficos no estilo matlab
import seaborn as sns #plotar gráficos com mais detalhes

# 1. Importar os arquivos

In [10]:
df_filmes = pd.read_csv ('data/movies.csv')
df_notas = pd.read_csv ('data/ratings.csv')

# 2. Analisar os datasets

## 2.1 Visualizar parte dos datasets

In [11]:
#Dataset sobre os filmes
df_filmes.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
#Dataset sobre nota dos filmes
df_notas.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## 2.2 Combinando os datasets

In [20]:
#Merge entre os datasets para visualizar os filmes vistos por cada usuário
filmes_notas = pd.merge(df_filmes, df_notas, on = 'movieId')
filmes_notas.head()


,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


## 2.3 Média de avaliação dos filmes

In [24]:
filmes_notas.groupby('title')['rating'].mean().sort_values( ascending = False ).head()

title
Gena the Crocodile (1969)              5.0
True Stories (1986)                    5.0
Cosmic Scrat-tastrophe (2015)          5.0
Love and Pigeons (1985)                5.0
Red Sorghum (Hong gao liang) (1987)    5.0
Name: rating, dtype: float64

## 2.4 Total de notas que cada filme recebeu
#### A nota média não é o ideal para cada filme, porque há filmes que há apenas uma nota atribuída para cada filme. portanto é necessário avaliar a quantidade de avaliações de cada filme

In [27]:
#Ver alguns filmes com nota 1
filmes_notas.groupby('title')['rating'].count().sort_values( ascending = True ).head()

title
'71 (2014)                                                                        1
Latter Days (2003)                                                                1
Late Shift, The (1996)                                                            1
Late Night with Conan O'Brien: The Best of Triumph the Insult Comic Dog (2004)    1
Late Night Shopping (2001)                                                        1
Name: rating, dtype: int64

In [29]:
#Ver os filmes com maior quantidade de avaliações 
filmes_notas.groupby('title')['rating'].count().sort_values( ascending = False ).head()

title
Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: rating, dtype: int64